In [2]:
%pip install -e 'src/hear21passt'

Obtaining file:///root/data/exploratory_notebooks/da/src/hear21passt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 23.6 MB/s eta 0:00:00
  Running setup.py develop for hear21passt
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 28.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [3]:
import sys
import os
import datetime
import re

import numpy as np
import pandas as pd
import torch
import torchaudio
import tqdm

from torch.optim import AdamW, lr_scheduler
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ChainDataset

from sklearn.metrics import f1_score

from IPython.display import Audio

from optim.lr_scheduler import get_scheduler_lambda
from utils.model import get_pretrained_passt_model
from utils.gdsc22_dataset import BuzzIterableDataset, BuzzAugmentedIterableDataset
from utils.dataset_augmentations import BuzzIterableTransformedDataset, MixUp1Level, MixUp2Level, RandomGain, Rolling, ColoredNoise

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 1. Initial configuration

In [4]:
torch.manual_seed(42)
np.random.seed(42)

In [5]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_processed_dir = os.path.join(project_dir, 'gdsc_data/data_processed/')
data_source_dir = os.path.join(project_dir, 'gdsc_data/data_source/')

In [6]:
MODEL_NAME = f"gdsc22_passt_kd_explr_t40_f4_45epochs_augmented_train@{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}.pt"
TORCH_SAVE_PATH = os.path.join(project_dir, f'checkpoints/{MODEL_NAME}')

TENSORBOARD_RUNS_PATH = os.path.join(project_dir, "torch_runlogs/")
TENSORBOARD_EXP_FOLDER = f"PaSST_kd_EXP_lr_mismatch_size_45epochs_augmented_train@{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
TENSORBOARD_EXP_COMMENT = "PaSST_s_kd_p16_128_ap486_EXP_lr_augmented_train"

TRAIN_BATCH_SIZE = 30
VAL_BATCH_SIZE = 30

# fine tuning and ohe of real labels
CLASSIFIER_N_CLASSES = 66

DATA_TAG = "data_8c86715"

In [7]:
#final_dataset_path = find_final_dataset_path(data_processed_dir, DATA_TAG)
final_dataset_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '01_apply_ir_function')
final_dataset_val_as_train_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '03_apply_ir_function_on_validation')

final_dataset_path = data_source_dir
final_dataset_val_as_train_path = os.path.join(data_processed_dir, DATA_TAG, '02_resample_validation/')

In [8]:
train_dir_augmented = os.path.join(final_dataset_path_augmented, 'train/')
val_as_train_dir_augmented = os.path.join(final_dataset_val_as_train_path_augmented, 'train/')

train_dir = os.path.join(final_dataset_path, 'train/')
val_as_train_dir = os.path.join(final_dataset_val_as_train_path, 'train/')

val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')

## 2. Dataloaders and batching

In [9]:
train_dataset = BuzzIterableDataset(
    train_dir,
    metadata_csv=os.path.join(train_dir, 'metadata.csv'), 
    shuffle=True,
    subsample=1.0,
    total_classes=CLASSIFIER_N_CLASSES
)
val_as_train_dataset = BuzzIterableDataset(
    val_as_train_dir,
    metadata_csv=os.path.join(val_as_train_dir, 'metadata.csv'), 
    shuffle=True,
    subsample=1.0,
    total_classes=CLASSIFIER_N_CLASSES
)

train_dataset_augmented = BuzzAugmentedIterableDataset(
    train_dir_augmented, 
    metadata_csv=os.path.join(final_dataset_path_augmented, 'metadata_grouped_train.csv'), 
    shuffle=True,
    subsample=0.2,
    total_classes=CLASSIFIER_N_CLASSES
)
val_as_train_dataset_augmented = BuzzAugmentedIterableDataset(
    val_as_train_dir_augmented, 
    metadata_csv=os.path.join(final_dataset_val_as_train_path_augmented, 'metadata_grouped_train.csv'), 
    shuffle=True,
    subsample=0.2,
    total_classes=CLASSIFIER_N_CLASSES
)
    
train_dataloader = DataLoader(
    ChainDataset([train_dataset, train_dataset_augmented, val_as_train_dataset, val_as_train_dataset_augmented]), 
    batch_size=TRAIN_BATCH_SIZE, 
    pin_memory=True
)

In [14]:
a, b, c, d = next(iter(train_dataloader))

In [17]:
d.argmax(dim=1)

tensor([38, 38, 38, 33, 33, 33, 33, 56, 61, 61, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19])

In [9]:
val_dataset = BuzzIterableDataset(val_dir, metadata_csv=os.path.join(val_dir, 'metadata.csv'))
val_dataloader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE)

## 3. Init pytorch objects

In [10]:
writer = SummaryWriter(os.path.join(TENSORBOARD_RUNS_PATH, TENSORBOARD_EXP_FOLDER), comment=TENSORBOARD_EXP_COMMENT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
%%sh
nvidia-smi

Tue Jul 11 09:39:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    25W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
# Default stride for t and f is 10
# NOTE: FMAX is used in frequency augmentation. Every sample has a random subset of fmin and fmax applied to it...
model = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=40, s_patchout_f=4)

#model = get_pretrained_passt_model(arch='passt_s_swa_p16_128_ap476', mode="all", n_classes=66, s_patchout_t=10, s_patchout_f=3)
model.to(device)



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

PaSSTGDSCIface(
  (mel): AugmentMelSTFT(
    winsize=800, hopsize=320
    (freqm): FrequencyMasking()
    (timem): TimeMasking()
  )
  (net): PaSST(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=307

In [ ]:
adamw = AdamW(model.net.parameters(), lr=0.00001, weight_decay=0.0001)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(adamw, get_scheduler_lambda())

print(f"Initial Learning rate:{lr_scheduler.get_last_lr()[0]}")
for epoch in range(0, 45):
    y_tr_pred_f1 = []
    y_tr_true_f1 = []
    train_loss = 0.0

    model.train()
    for _, _, batch, y in (progress := tqdm.tqdm(train_dataloader)):
        adamw.zero_grad()
        
        batch = batch.to(device)
        y = y.to(device)
        y_hat, emb = model(batch)
        loss = F.cross_entropy(y_hat, y, reduction="none")
        loss = loss.sum()

        train_loss += float(loss)
        progress.set_description(f"TrainL {float(loss):.2f}")

        loss.backward()
        adamw.step()

        prob = F.softmax(y_hat, dim=1)
        y_pred = torch.argmax(prob, dim=1)

        y_tr_pred_f1.append(y_pred.cpu())
        y_tr_true_f1.append(y.cpu())

    #    print(f"Train Loss (on minibatch): {loss}",  end='\r')

    f_score_train = f1_score(torch.cat(y_tr_true_f1).numpy(), torch.cat(y_tr_pred_f1).numpy(), average='macro')
    print(f"=" * 80)
    print("\r\n")
    print(f"Train Loss(epoch={epoch}): {train_loss}")
    print(f"F1 score(train) {f_score_train}")
    print("\r\n")
    print(f"=" * 80)
    print("\r\n")

    val_loss = 0.0
    ypred_f1 = []
    ytrue_f1 = []
    model.eval()
    
    with torch.no_grad():
        for _, _, batch, y_val in tqdm.tqdm(val_dataloader):
            batch = batch.to(device)
            y_val = y_val.to(device)

            y_hat, emb = model(batch)
            _loss = F.cross_entropy(y_hat, y_val, reduction="none")

            prob = F.softmax(y_hat, dim=1)
            y_pred = torch.argmax(prob, dim=1)

            ypred_f1.append(y_pred.cpu())
            ytrue_f1.append(y_val.cpu())

            val_loss += _loss.sum()

    f1_score_val = f1_score(torch.cat(ytrue_f1).numpy(), torch.cat(ypred_f1).numpy(), average='macro')

    print(f"=" * 80)
    print("\r\n")
    print(f"Val Loss: {val_loss}")
    print(f"F1 score(val) {f1_score_val}")
    print("\r\n")    
    print(f"=" * 80)
    print("\r\n")
       
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('F1/train', f_score_train, epoch)
    writer.add_scalar('F1/val', f1_score_val, epoch)
    writer.add_scalar('LR', lr_scheduler.get_last_lr()[0], epoch)

    lr_scheduler.step()
    print(f"New lr: {lr_scheduler.get_last_lr()[0]}")
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': adamw.state_dict(),
        }, TORCH_SAVE_PATH)

Initial Learning rate:1.7422374639493516e-07


0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([30, 1, 128, 1000])
self.norm(x) torch.Size([30, 768, 12, 99])
 patch_embed :  torch.Size([30, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X Before time Patchout of 40  torch.Size([30, 768, 12, 99])
X after time Patchout torch.Size([30, 768, 12, 59])
X Before Freq Patchout of 4  torch.Size([30, 768, 12, 59])
 
 X after freq Patchout:  torch.Size([30, 768, 8, 59])
X flattened torch.Size([30, 472, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([30, 1, 768])
 self.dist_token.shape torch.Size([30, 1, 768])
 final sequence x torch.Size([30, 474, 768])
 after 12 atten blocks x torch.Size([30, 474, 768])
forward_features torch.Size([30, 768])
head torch.Size([30, 66])


TrainL 74.30: : 457it [19:48,  2.60s/it] 




Train Loss(epoch=0): 57697.97215270996
F1 score(train) 0.006071273303369849






...

0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
TrainL 100.47: : 457it [19:57,  2.62s/it]




Train Loss(epoch=5): 31027.02473115921
F1 score(train) 0.25498199696167667






0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
17it [00:47,  2.82s/it]




Val Loss: 951.299072265625
F1 score(val) 0.2421370822834725




New lr: 1e-05


0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
TrainL 76.06: : 82it [03:43,  2.71s/it]

In [ ]:
model_eval = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=0, s_patchout_f=0)
state = torch.load(TORCH_SAVE_PATH)
model_eval.load_state_dict(state['model_state_dict'])

In [7]:
model_eval = model_eval.cuda()

In [8]:
    val_loss = 0.0
    ypred_f1 = []
    ytrue_f1 = []
    model_eval.eval()
    
    with torch.no_grad():
        for batch, y_val in tqdm.tqdm(val_dataloader):
            batch = batch.cuda()
            y_val = y_val.cuda()

            y_hat, emb = model_eval(batch)
            _loss = F.cross_entropy(y_hat, y_val, reduction="none")

            prob = F.softmax(y_hat, dim=1)
            y_pred = torch.argmax(prob, dim=1)

            ypred_f1.append(y_pred.cpu())
            ytrue_f1.append(y_val.cpu())

            val_loss += _loss.sum()

    f1_score_val = f1_score(torch.cat(ytrue_f1).numpy(), torch.cat(ypred_f1).numpy(), average='macro')

    print(f"Val Loss: {val_loss}")
    print(f"=" * 80)
    print(f"F1 score(val) {f1_score_val}")
    print(f"=" * 80)

  0%|          | 0/169 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([12, 1, 128, 1000])
self.norm(x) torch.Size([12, 768, 12, 99])
 patch_embed :  torch.Size([12, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X flattened torch.Size([12, 1188, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([12, 1, 768])
 self.dist_token.shape torch.Size([12, 1, 768])
 final sequence x torch.Size([12, 1190, 768])
 after 12 atten blocks x torch.Size([12, 1190, 768])
forward_features torch.Size([12, 768])
head torch.Size([12, 66])


100%|██████████| 169/169 [03:09<00:00,  1.12s/it]

Val Loss: 1069.6846923828125
F1 score(val) 0.8392231749961314


In [10]:
%%sh
nvidia-smi

Thu Jul  6 18:38:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   43C    P0    28W /  70W |  15010MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## Miscellaneous

### Glossary

SWA - Stochastic weight averaging

### Model stored weights
```
Downloading: "https://github.com/kkoutini/PaSST/releases/download/v0.0.1-audioset/passt-s-f128-p16-s10-ap.476-swa.pt" to /root/.cache/torch/hub/checkpoints/passt-s-f128-p16-s10-ap.476-swa.pt
```